# Actividad Integradora

## Installation

In [368]:
!pip install mesa

## Libraries

In [369]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

#Other libraries
import numpy as np
import pandas as pd
import random
import time
import datetime

## Classes

In [423]:
color_map = {
    0: -0,
    1: 0.75,
    2: -1,
    3: 1
}

def get_grid(model):
    """We need to generate the room which is a MxN grid"""
    grid = np.zeros((model.grid.width, model.grid.height))
    for (content, x, y) in model.grid.coord_iter():
        for data_content in content: 
          grid[x][y] = color_map[data_content.state]
    return grid
    

class Person(Agent):
    """ Agent representing a person
        0 = no person
        1 = alive 
        2 = zombie
        3 = safe
     """

    def __init__(self, pos,  model, state):
        super().__init__(pos, model)
        self.x, self.y = pos
        self.nextPos = None
        self.moves = 0
        self.state = state

    def spread(self):
      if self.state != 2:
        return
      neighbors = self.model.grid.get_neighbors(self.pos,moore = False, include_center=True)
      for neighbor in neighbors: 
          if neighbor.pos == self.pos and neighbor.state != 3:
            neighbor.state = 2

    def save(self):
      if self.state != 1:
        return
      neighbors = self.model.grid.get_neighborhood(
            self.pos,
            moore=False,
            include_center=False)
      if len(neighbors) < 4:
        self.state = 3

    def move(self):
      if self.state == 3 or self.state == 0:
        return
      possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
      nextPos = self.random.choice(possible_steps)
      self.nextPos = nextPos 
      self.model.grid.move_agent(self, nextPos)

    
    def step(self):
      # The agent's step will go here.
      self.save()
      self.move()
      self.save()
      self.spread()


class SurvivalModel(Model):
  """A game of life model with some number of cells (agents)."""
  def __init__(self, M, N, total_alive, total_zombies):
    self.total_alive = total_alive
    self.total_zombies = total_zombies
    self.total_steps = 0
    self.grid = MultiGrid(M, N, False) 
    self.schedule = SimultaneousActivation(self)
    self.all_agents = []

    cells = list(self.grid.empties)

    for i in range(total_alive):
      rand_cell = random.choice(cells)
      cell = Person(rand_cell, self, 1)
      self.all_agents.append(cell)
      self.grid.place_agent(cell,rand_cell)
      self.schedule.add(cell)
      cells.remove(rand_cell)
    
    for i in range(total_zombies):
      rand_cell = random.choice(cells)
      cell = Person(rand_cell, self, 2)
      self.all_agents.append(cell)
      self.grid.place_agent(cell,rand_cell)
      self.schedule.add(cell)
      cells.remove(rand_cell)

    others = M*N - total_zombies - total_alive
    ''' 
    for i in range(others):
      rand_cell = random.choice(cells)
      cell = Person(rand_cell, self, 0)
      self.grid.place_agent(cell,rand_cell)
      self.schedule.add(cell)
      cells.remove(rand_cell) '''

    self.datacollector = DataCollector(
        model_reporters = {'Grid': get_grid},
        agent_reporters={'Moves': lambda a: getattr(a, 'moves', None)}
    )

  def status(self):
    out = [0,0,0,0]
    for agent in self.all_agents:
      out[agent.state] += 1
    return out

  def step(self):
    self.datacollector.collect(self)
    self.schedule.step()
    self.total_steps += 1



## Ejemplo 1

In [447]:
M = 25
N = 25
max_steps = 30
total_zombies = 20
total_alive = 50
step_counter = 0

model = SurvivalModel(M, N, total_alive, total_zombies)

status_ini = model.status()

while step_counter < max_steps and model.status()[1] > 0:
  model.step()
  step_counter += 1

status_fin = model.status()

print(status_ini)
print(status_fin)
print("Se realizaron ", model.total_steps, " steps")
print("La razón de cambio de personas no infectadas es de ", (1 - (status_fin[1]+status_fin[3])/total_alive) * 100, " %")

[0, 50, 20, 0]
[0, 7, 41, 22]
Se realizaron  30  steps
La razón de cambio de personas no infectadas es de  42.00000000000001  %


In [448]:
all_grid = model.datacollector.get_model_vars_dataframe()


In [449]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='PiYG', vmin=-1, vmax=1)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [450]:
anim = animation.FuncAnimation(fig, animate, frames=max_steps)

anim

## Ejemplo 2

In [455]:
M = 20
N = 20
max_steps = 30
total_zombies = 20
total_alive = 30
step_counter = 0

model = SurvivalModel(M, N, total_alive, total_zombies)

status_ini = model.status()

while step_counter < max_steps and model.status()[1] > 0:
  model.step()
  step_counter += 1

status_fin = model.status()

print(status_ini)
print(status_fin)
print("Se realizaron ", model.total_steps, " steps")
print("La razón de cambio de personas no infectadas es de ", (1 - (status_fin[1]+status_fin[3])/total_alive) * 100, " %")

[0, 30, 20, 0]
[0, 1, 32, 17]
Se realizaron  30  steps
La razón de cambio de personas no infectadas es de  40.0  %


In [456]:
all_grid = model.datacollector.get_model_vars_dataframe()


In [457]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='PiYG', vmin=-1, vmax=1)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [458]:
anim = animation.FuncAnimation(fig, animate, frames=max_steps)

anim

## Ejemplo 3

In [477]:
M = 10
N = 10
max_steps = 10
total_zombies = 1
total_alive = 5
step_counter = 0

model = SurvivalModel(M, N, total_alive, total_zombies)

status_ini = model.status()

while step_counter < max_steps and model.status()[1] > 0:
  model.step()
  step_counter += 1

status_fin = model.status()

print(status_ini)
print(status_fin)
print("Se realizaron ", model.total_steps, " steps")
print("La razón de cambio de personas no infectadas es de ", (1 - (status_fin[1]+status_fin[3])/total_alive) * 100, " %")

[0, 5, 1, 0]
[0, 2, 1, 3]
Se realizaron  10  steps
La razón de cambio de personas no infectadas es de  0.0  %


In [478]:
all_grid = model.datacollector.get_model_vars_dataframe()


In [479]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='PiYG', vmin=-1, vmax=1)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [480]:
anim = animation.FuncAnimation(fig, animate, frames=max_steps)

anim